In [ ]:
pip install python-dotenv groq pydantic

In [5]:
import os
import json
from groq import Groq
from typing import Literal
from dotenv import load_dotenv
from pydantic import BaseModel

load_dotenv()

api_key = os.getenv("GROQ_API_KEY")
if not api_key:
    raise ValueError("GROQ_API_KEY not found. Set it as an environment variable.")

client = Groq(api_key=api_key)

In [ ]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant."
        },
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama-3.3-70b-versatile"
)

print(chat_completion.choices[0].message.content)

In [8]:
class ProductReview(BaseModel):
    rating: float
    product_name: str
    key_features: list[str]
    sentiment: Literal["positive", "negative", "neutral"]

response = client.chat.completions.create(
    model="moonshotai/kimi-k2-instruct",
    messages=[
        {"role": "system", "content": "Extract product review information from the text."},
        {
            "role": "user",
            "content": "I bought the UltraSound Headphones last week and I'm really impressed! The noise cancellation is amazing and the battery lasts all day. Sound quality is crisp and clear. I'd give it 4.5 out of 5 stars.",
        },
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "product_review",
            "schema": ProductReview.model_json_schema()
        }
    }
)

review = ProductReview.model_validate(json.loads(response.choices[0].message.content))
print(json.dumps(review.model_dump(), indent=2))

{
  "rating": 4.5,
  "product_name": "UltraSound Headphones",
  "key_features": [
    "amazing noise cancellation",
    "all-day battery life",
    "crisp and clear sound quality"
  ],
  "sentiment": "positive"
}
